In [19]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
from pandas import json_normalize
from IPython.display import Image
from IPython.core.display import HTML
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
load_dotenv()

True

In [2]:
# Function to remove undesired columns

def remove_columns(df, column_name):
    
    """
    This is a function that removes undesired columns. Requires two arguments.
    Arguments: dataframe, column name
    Input: the current dataframe
    Output: the current dataframe without the selected columns
    """
    
    df.drop(columns=f"{column_name}", inplace=True)
    
    return df.sample(2)

In [3]:
# Function to rename columns

def rename_columns(df, old_name, new_name):
    
    """
    This a functions that renames the name of any given columns. Requires three arguments.
    Arguments: dataframe, old name of the column, new name of the column.
    Input: the current column name
    Output: the column renamed
    """
    
    df.rename(columns={f"{old_name}": f"{new_name}"}, inplace=True)
    return df.sample(2)

In [4]:
# Function to clean the atmosphere column

def clean_atmosphere(df,column_name,string,replacement):
    
    """
    This a function that cleans the Atmospheric opacity columns by replacing its elements.
    Requires three arguments.
    Arguments: dataframe, column name, string to replace, new string.
    Input: any string
    Output: a string
    """
    
    df[f"{column_name}"] = df[f"{column_name}"].replace(f"{string}",f"{replacement}")    
    return df.sample(2)

In [5]:
# Function to clean the month column

def clean_month(df,column_name):
    
    """
    This is a function that cleans the Month column. Requires two arguments. Removes unwanted strings and only keeps the value
    Arguments: dataframe, column name
    Input: string + digit
    Output: digit
    """
    
    df[f"{column_name}"] = df[f"{column_name}"].str.extract(r"(\d)")    
    return df.sample(2)

In [6]:
# Function to convert degrees Fahrenheit to degrees Celsius

def FtoC(df, column_name):
    
    """
    This is a function that converts any temperature in degrees Fahrenheit to degrees Celsius.
    Requires two arguments.
    Arguments: dataframe, column name
    Input: an INTEGER in Fahrenheit degrees
    Output: an INTEGER in Celsius degrees
    """
    
    df[f"{column_name}"] = [((i - 32.0) * 5.0/9.0) for i in df[f"{column_name}"]]

    return df.sample(2)

In [7]:
# Function to convert milibar to Pascals

def mbartoPa(df, column_name):
    
    """
    This is a function that converts any temperature in degrees Fahrenheit to degrees Celsius.
    Requires two arguments.
    Arguments: dataframe, column name
    Input: an INTEGER in Fahrenheit degrees
    Output: an INTEGER in Celsius degrees
    """
    
    df[f"{column_name}"] = [i*100 for i in df[f"{column_name}"]]

    return df.sample(2)

In [8]:
# Function to round decimals from floats

def roundval(df, column_name, n):
    """
    This is a function that rounds any float to a given n value. Requires three arguments.
    Arguments: dataframe, name of the column where you want to rewrite the values, value of the round
    Input: a float with multiple decimals
    Output: a float with n decimals
    """
    df[f"{column_name}"] = [round(i,n) for i in df[f"{column_name}"]]
    return df.sample(2)

In [22]:
def floatify(df, column_name):
    """
    This is a function that floats any value. Requires two arguments.
    Arguments: dataframe, name of the column where you want to rewrite the values
    Input: a string or integer
    Output: a float
    """
    df[f"{column_name}"] = df[f"{column_name}"].astype(float)
    return df.sample(2)

In [9]:
# This is a function to call NASA api with a given a specific date and Camera type.

def call_Curiosity (date, camera):
    """
    This is a function that calls NASA API 'Mars Rover Photos' with two arguments. It returns the url from
    a specific camera onboard Curiosity rover.
    date: input the desired date in the format YYYY-MM-DD as a STRING,
    camera: select between FHAZ, RHAZ, MAST, CHEMCAM, MAHLI, MARDI, NAVCAM, PANCAM, MINITES, as STRING
    
    """
        
    try:
        nasa = os.getenv("token")
        url = f"https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos?earth_date={date}&camera={camera}&api_key={nasa}"
        request = requests.get(url)
        df = pd.DataFrame(request.json())
        df_clean = pd.DataFrame(df.values[0][0])
        image_url = list(df_clean["img_src"])[0]
        display(Image(image_url, width=300, height=200))
        
        return f"Image available for camera {camera} onboard Curiosity rover"

    except:
            
        return f"No image available on {date} for camera {camera} onboard Curiosity rover, please select another date"

In [10]:
def get_pictures_Curiosity(date):
    
    """
    This is a function that calls call_NASA function with one argument. It returns the url of all the pictures 
    taken by all the cameras of Curiosity rover from a specific Sol date.
    date: input the desired date in the format YYYY-MM-DD as STRING.
    
    """       
    cameralist = ["FHAZ", "RHAZ", "MAST", "CHEMCAM", "MAHLI", "MARDI", "NAVCAM", "PANCAM", "MINITES"]
    for i in cameralist:
        print(call_Curiosity(date, i))
    pass

In [15]:
def predict():
    pass

In [20]:
def cool_plots():
    fig = make_subplots(rows=3, cols=2, 
                        subplot_titles=("Temperature cycle of Mars", "Temperature cycle of Papua New Guinea",
                                        "Pressure cycle of Mars", "Pressure cycle of Papua New Guinea",
                                       "Pressure vs Temperature on Mars",
                                       "Pressure vs Temperature on Papua New Guinea"))

    # Add traces
    fig.add_trace(
        go.Line(x=mars["Earth Date"], y=mars["Mean_temp"], name="Temp Mars"),
        row=1, col=1)
    fig.add_trace(
        go.Line(x=earth["Earth Date"], y=earth["Mean_temp"], name="Temp Earth"),
        row=1, col=2)
    fig.add_trace(
        go.Line(x=mars["Earth Date"], y=mars["Pressure"], name="Pressure Mars"),
        row=2, col=1)
    fig.add_trace(
        go.Line(x=earth["Earth Date"], y=earth["Pressure"], name="Pressure Earth"),
        row=2, col=2)
    fig.add_trace(
        go.Scatter(x=mars["Mean_temp"], y=mars["Pressure"], mode="markers", name="Temp vs Pres Mars"),
        row=3, col=1)
    fig.add_trace(
        go.Scatter(x=earth["Mean_temp"], y=earth["Pressure"], mode="markers", name="Temp vs Pres Earth"),
        row=3, col=2)

    # Update xaxis properties
    fig.update_xaxes(title_text="Years", row=1, col=1)
    fig.update_xaxes(title_text="Years", row=1, col=2)
    fig.update_xaxes(title_text="Years", row=2, col=1)
    fig.update_xaxes(title_text="Years", row=2, col=2)
    fig.update_xaxes(title_text="Temperature (°C)", row=3, col=1)
    fig.update_xaxes(title_text="Temperature (°C)", row=3, col=2)

    # Update yaxis properties
    fig.update_yaxes(title_text="Temperature (°C)", row=1, col=1)
    fig.update_yaxes(title_text="Temperature (°C)", row=1, col=2)
    fig.update_yaxes(title_text="Pressure (Pa)", row=2, col=1)
    fig.update_yaxes(title_text="Pressure (Pa)", row=2, col=2)
    fig.update_yaxes(title_text="Pressure (Pa)", row=3, col=1)
    fig.update_yaxes(title_text="Pressure (Pa)", row=3, col=2)

    # Update title and height
    fig.update_layout(height=1100, width=1000, title_text="Summary of surface conditions on Mars and Papua New Guinea")
    
    return fig.show()